In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-summary/news_summary_more.csv
/kaggle/input/news-summary/news_summary.csv


In [2]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", 

In [3]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 81.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1f0958b4692647972e06d553ece43890b5dae382f6f21654bf46cf0c3f1b9fa4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [5]:

from datasets import load_dataset
import evaluate
from transformers import BartForConditionalGeneration, BartTokenizerFast
from transformers import Trainer, TrainingArguments

# 1. Load dataset
dataset = load_dataset("csv", data_files={"dataset": "/kaggle/input/news-summary/news_summary.csv"}, encoding="ISO-8859-1")
dataset = dataset['dataset'].train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']

# 2. Load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 3. Tokenization function
def tokenize(batch):
    # Convert everything to string to avoid None or other types
    inputs = tokenizer([str(x) for x in batch['ctext']], 
                       max_length=512, truncation=True, padding="max_length")
    targets = tokenizer([str(x) for x in batch['text']], 
                        max_length=128, truncation=True, padding="max_length")
    
    batch['input_ids'] = inputs['input_ids']
    batch['attention_mask'] = inputs['attention_mask']
    batch['labels'] = targets['input_ids']
    return batch

# Apply tokenization
remove_cols = ['author', 'date', 'headlines', 'read_more', 'ctext', 'text']
tokenized_train = train_dataset.map(tokenize, batched=True, remove_columns=remove_cols)
tokenized_val = val_dataset.map(tokenize, batched=True, remove_columns=remove_cols)


# 4. Training arguments compatible with older transformers
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    learning_rate=3e-5,
    fp16=True,
    logging_strategy="epoch",     # <-- log at the end of each epoch
    save_strategy="epoch",        # <-- save checkpoint at end of each epoch
    save_total_limit=2,
    report_to="none"
)

# 5. ROUGE metric
rouge = evaluate.load("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    result = rouge.compute(predictions=pred_str, references=labels_str)
    return {key: value * 100 for key, value in result.items()}

# 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 7. Train
trainer.train()

# 8. Save
model.save_pretrained("./fine_tuned_bart_large_cnn")
tokenizer.save_pretrained("./fine_tuned_bart_large_cnn")

print("Fine-tuning complete!")

2025-09-27 08:27:17.067564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758961637.266743      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758961637.326986      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating dataset split: 0 examples [00:00, ? examples/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/452 [00:00<?, ? examples/s]

/tmp/ipykernel_36/1327848807.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
4062,0.987000
8124,0.601700
12186,0.361100


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Fine-tuning complete!


In [6]:
# Manual ROUGE evaluation (works regardless of transformers version)
import torch
from transformers import DataCollatorForSeq2Seq
import evaluate
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tune this for memory; 4 is usually safe on Kaggle T4
eval_batch_size = 4

# collator pads to longest in batch and prepares tensors expected by model.generate
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest", return_tensors="pt")

# Create a DataLoader over the tokenized validation dataset
eval_dataloader = DataLoader(tokenized_val, batch_size=eval_batch_size, collate_fn=data_collator)

preds = []
refs = []

model.eval()
with torch.no_grad():
    for batch in eval_dataloader:
        # move tensors to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # generate summaries
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=130,
            min_length=30,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True,
            no_repeat_ngram_size=3
        )

        # decode predictions
        batch_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        preds.extend(batch_preds)

        # prepare references: replace -100 with pad_token_id, then decode
        labels = batch["labels"].detach().cpu().numpy()
        labels[labels == -100] = tokenizer.pad_token_id
        batch_refs = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        refs.extend(batch_refs)

# compute ROUGE
rouge = evaluate.load("rouge")
result = rouge.compute(predictions=preds, references=refs)
result = {k: v * 100 for k, v in result.items()}   # percentage
print("ROUGE results (percent):", result)

# show 5 example pairs to inspect quality
print("\nExamples (predicted -> reference):\n")
for i in range(5):
    print(f"Pred {i+1}: ", preds[i][:400])
    print(f"Ref  {i+1}: ", refs[i][:400])
    print("-" * 80)


ROUGE results (percent): {'rouge1': 49.867936895514795, 'rouge2': 27.5531305031589, 'rougeL': 37.26528886016578, 'rougeLsum': 37.22370285410269}

Examples (predicted -> reference):

Pred 1:  Actor Dileep has been arrested in connection to the Malayalam actress abduction case. He was interrogated along with director Nadirshah for 13 hours to understand his involvement in the case, and his wife Kavya Madhavan's place of business was raided as well. Notably, actress Priyanka Chopra was abducted from the sets of her film Georgettan's Pooram in 2014.
Ref  1:  Actor Dileep was arrested on Monday in connection with the abduction and assault case of Malayalam actress, which took place in February 2017. Earlier, Dileep got linked to the case after his name was mentioned in a letter of a jail inmate, who shared the cell with the prime accused Pulsar Suni. Dileep has claimed that he doesn't know Suni.
--------------------------------------------------------------------------------
Pred 2:  Austra

In [7]:
from transformers import BartForConditionalGeneration, BartTokenizerFast
import torch

# Load the saved model
model_path = "./fine_tuned_bart_large_cnn"
tokenizer = BartTokenizerFast.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)
model.eval()  # Set model to evaluation mode


/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [14]:
def generate_summary(text, max_input_len=512, max_output_len=128):
    # Tokenize input
    inputs = tokenizer(
        text,
        max_length=max_input_len,
        truncation=True,
        return_tensors="pt"
    )

    # Generate summary IDs
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=max_output_len,
        min_length=30,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode output properly
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [23]:
def generate_summary(text, max_input_len=512, max_new_tokens=128, min_output_len=10):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Tokenize input and move to same device
    inputs = tokenizer(
        text,
        max_length=max_input_len,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    # Generate summary with relaxed settings
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_new_tokens=max_new_tokens,
        min_length=min_output_len,
        length_penalty=1.0,       # less aggressive than 2.0
        early_stopping=False,     # safer for short texts
        no_repeat_ngram_size=2    # avoid repeating phrases
    )

    # Decode
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary.strip() if summary else "⚠️ No summary generated"


In [26]:
article = """
Scientists at the European Organization for Nuclear Research (CERN) have announced a breakthrough in particle physics after completing new experiments at the Large Hadron Collider (LHC). The experiments aimed to explore the behavior of subatomic particles under extreme conditions, providing insights into the fundamental forces of the universe. Researchers reported observing rare particle interactions that were previously theoretical, potentially shedding light on unanswered questions in quantum mechanics and the Standard Model of particle physics.

The LHC, the world’s largest and most powerful particle accelerator, has been operational since 2008 and continues to provide unprecedented data for physicists around the globe. In the latest experiments, protons were accelerated to near-light speeds and collided, producing a range of exotic particles, some of which have never been detected before. These findings could have implications for understanding dark matter, antimatter, and the early moments of the universe following the Big Bang.

CERN scientists emphasize that while these results are preliminary, they offer promising avenues for future research. International teams are already planning follow-up experiments to replicate and expand upon these observations. Beyond the theoretical implications, the technologies developed for these experiments are expected to benefit fields like medical imaging, materials science, and information technology. The scientific community is eagerly awaiting peer-reviewed publications to validate these groundbreaking discoveries.

"""

summary = generate_summary(article)
print("Generated Summary:\n", summary)


Generated Summary:
 Scientists at the European Organization for Nuclear Research (CERN) have announced a breakthrough in particle physics after completing new experiments at its Large Hadron Collider (LHC). The experiments aimed to explore the behavior of subatomic particles under extreme conditions, providing insights into the fundamental forces of the universe. Researchers reported observing rare particle interactions that were previously theoretical, potentially shedding light on unanswered questions in quantum mechanics.
